In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --------------------------------------------------------
# Squeeze-and-Excitation Layer
# --------------------------------------------------------
class SELayer(nn.Module):
    def __init__(self, in_channels, reduction=4):
        super().__init__()
        hidden = in_channels // reduction
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),       # -> [B, C, 1, 1]
            nn.Conv2d(in_channels, hidden, 1),
            nn.SiLU(),
            nn.Conv2d(hidden, in_channels, 1),
            nn.Sigmoid())

    def forward(self, x):
        scale = self.fc(x)
        return x * scale

In [ ]:
# --------------------------------------------------------
# MBConv Block
# --------------------------------------------------------
class MBConv(nn.Module):
    def __init__(self, in_ch, out_ch, expand_ratio, stride, kernel):
        super().__init__()

        hidden = in_ch * expand_ratio
        self.use_residual = (stride == 1 and in_ch == out_ch)

        layers = []

        # 1) Expansion (1x1 conv)
        if expand_ratio != 1:
            layers += [
                nn.Conv2d(in_ch, hidden, 1, bias=False),
                nn.BatchNorm2d(hidden),
                nn.SiLU()
            ]

        # 2) Depthwise conv
        layers += [
            nn.Conv2d(hidden, hidden, kernel, stride=stride,
                      padding=kernel//2, groups=hidden, bias=False),
            nn.BatchNorm2d(hidden),
            nn.SiLU()
        ]

        # 3) SE
        layers += [SELayer(hidden)]

        # 4) Projection (1x1 conv)
        layers += [
            nn.Conv2d(hidden, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch),
        ]

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        out = self.block(x)
        if self.use_residual:
            return x + out
        return out


In [ ]:

class EfficientNetB0(nn.Module):
    def __init__(self):
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU()
        )

        config = [
            (1, 16, 1, 1, 3),
            (6, 24, 2, 2, 3),
            (6, 40, 2, 2, 5),
            (6, 80, 3, 2, 3),
            (6, 112, 3, 1, 5),
            (6, 192, 4, 2, 5),
            (6, 320, 1, 1, 3),
        ]

        layers = []
        in_channels = 32

        for expand, out_channels, repeats, stride, kernel in config:
            for i in range(repeats):
                s = stride if i == 0 else 1
                layers.append(MBConv(in_channels, out_channels, expand, s, kernel))
                in_channels = out_channels

        self.blocks = nn.Sequential(*layers)

        self.head = nn.Sequential(
            nn.Conv2d(320, 1280, 1),
            nn.BatchNorm2d(1280),
            nn.SiLU()
        )

        self.classifier = nn.Linear(1280, 1000)

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = torch.flatten(x, 1)
        return self.classifier(x)